# Load Dataset

In [1]:
import glob
import random
import os

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms


def to_rgb(image):
    rgb_image = Image.new("RGB", image.size)
    rgb_image.paste(image)
    return rgb_image


class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, f"{mode}A") + "/*.*"))
        self.files_B = sorted(glob.glob(os.path.join(root, f"{mode}B") + "/*.*"))

    def __getitem__(self, index):
        image_A = Image.open(self.files_A[index % len(self.files_A)])

        if self.unaligned:
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

        # Convert grayscale images to rgb
        if image_A.mode != "RGB":
            image_A = to_rgb(image_A)
        if image_B.mode != "RGB":
            image_B = to_rgb(image_B)

        item_A = self.transform(image_A)
        item_B = self.transform(image_B)
        return {"A": item_A, "B": item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))


# Load Utils

In [2]:
import time
import datetime
import sys

from torch.autograd import Variable
import torch
import numpy as np

from torchvision.utils import save_image


class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0, 1) > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))


class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)


# Build Model

## Library

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch

## Weight Init

In [4]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, "bias") and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

## Residual Block

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), # 양옆 위 아래로 padding해주는 기능
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features), # Instance 단위로 Normalization하는 방법, batch, channel과 무관하게 normalization (1,H,W)에 대해서 Norm
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )

    def forward(self, x):
        return x + self.block(x)

## Generator

In [6]:
class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks): # residual block을 몇 개 만들지에 대한 지표
        super(GeneratorResNet, self).__init__() 

        channels = input_shape[0] # channel 정보

        # Initial convolution block
        out_features = 64 # 64x64 이미지를 만들예정
        model = [
            nn.ReflectionPad2d(channels), # 1 Dim 이미지를 3 Dim 으로 확장
            nn.Conv2d(channels, out_features, 7), # 3차원 -> 64차원, kernel = 7
            nn.InstanceNorm2d(out_features), # Instance 단위 Normalization
            nn.ReLU(inplace=True), # ReLU
        ]
        in_features = out_features

        # Downsampling
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1), # 모델을 2배로 늘려가며 추가 // 64 -> 128 channel, 128 -> 256 channel
                nn.InstanceNorm2d(out_features), 
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(out_features)] # 이후에 residual block 추가

        # Upsampling
        for _ in range(2):
            out_features //= 2 # 256 -> 128차원, 128 -> 64차원
            model += [
                nn.Upsample(scale_factor=2), # multiplier for spatial size. 
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1), # 256 -> 128, 128-> 64
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Output layer
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

## Discriminator

In [7]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        channels, height, width = input_shape

        # Calculate output shape of image discriminator (PatchGAN)
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img):
        return self.model(img)


# Train Model

## Library

In [8]:
import argparse
import os
import numpy as np
import math
import itertools
import datetime
import time

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

## Arguments

In [9]:
data_name = 'monet2photo'

cuda = torch.cuda.is_available()
lr = 0.0002
b1 = 0.5
b2 = 0.999

batch_size = 2
total_epoch = 50

## Setting Environment

In [10]:
os.makedirs(f"images/{data_name}", exist_ok=True)
os.makedirs(f"saved_models/{data_name}", exist_ok=True)
now_epoch = len(glob.glob(f'saved_models/{data_name}/D_B_*')) -1 

# Losses
criterion_GAN = torch.nn.MSELoss().cuda()
criterion_cycle = torch.nn.L1Loss().cuda()
criterion_identity = torch.nn.L1Loss().cuda()

# Image Info
input_shape = (3, 256, 256)

# Set Model
G_AB = GeneratorResNet(input_shape, 9).cuda()
G_BA = GeneratorResNet(input_shape, 9).cuda()
D_A = Discriminator(input_shape).cuda()
D_B = Discriminator(input_shape).cuda()

if now_epoch != 0:
    # Load pretrained models
    G_AB.load_state_dict(torch.load("saved_models/%s/G_AB_%d.pth" % (data_name, now_epoch)))
    G_BA.load_state_dict(torch.load("saved_models/%s/G_BA_%d.pth" % (data_name, now_epoch)))
    D_A.load_state_dict(torch.load("saved_models/%s/D_A_%d.pth" % (data_name, now_epoch)))
    D_B.load_state_dict(torch.load("saved_models/%s/D_B_%d.pth" % (data_name, now_epoch)))
else:
    # Initialize weights
    G_AB.apply(weights_init_normal)
    G_BA.apply(weights_init_normal)
    D_A.apply(weights_init_normal)
    D_B.apply(weights_init_normal)

# set optimizer



optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr, betas=(b1, b2)
)
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(b1, b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(b1, b2))

# Learning rate update schedulers
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=LambdaLR(total_epoch, now_epoch, 15).step
)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_A, lr_lambda=LambdaLR(total_epoch, now_epoch, 15).step
)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_B, lr_lambda=LambdaLR(total_epoch, now_epoch, 15).step
)

Tensor = torch.cuda.FloatTensor

# Buffers of previously generated samples
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

transforms_ = [ 
    transforms.Resize(int(256 * 1.12), Image.BICUBIC), # 원본 이미지보다 조금 더 크게 resize하고, 남는 부분은 쌍선형 보간법을 이용해 보간해줌
    transforms.RandomCrop((256, 256)), # 이후에 데이터를 crop해주면, 좀 더 diverse한 이미지를 얻을 수 있음
    transforms.RandomHorizontalFlip(), # 수평 Flip도 적용해주고
    transforms.ToTensor(), # Tensor로도 변형해준다.
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

# Training data loader
dataloader = DataLoader(
    ImageDataset(f"../../data/{data_name}", transforms_=transforms_, unaligned=True),
    batch_size=batch_size,
    shuffle=True,
)
# Test data loader
val_dataloader = DataLoader(
    ImageDataset(f"../../data/{data_name}", transforms_=transforms_, unaligned=True, mode="test"),
    batch_size=5,
    shuffle=True,
)


In [11]:
def sample_images(batches_done):
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))
    G_AB.eval()
    G_BA.eval()
    real_A = Variable(imgs["A"].type(Tensor))
    fake_B = G_AB(real_A)
    real_B = Variable(imgs["B"].type(Tensor))
    fake_A = G_BA(real_B)
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)
    real_B = make_grid(real_B, nrow=5, normalize=True)
    fake_A = make_grid(fake_A, nrow=5, normalize=True)
    fake_B = make_grid(fake_B, nrow=5, normalize=True)
    # Arange images along y-axis
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)
    save_image(image_grid, f"images/{data_name}/{batches_done}.png", normalize=False)

## Train

In [12]:
prev_time = time.time()
for epoch in range(now_epoch, total_epoch):
    for i, batch in enumerate(dataloader):

        # Set model input
        real_A = Variable(batch["A"].type(Tensor))
        real_B = Variable(batch["B"].type(Tensor))

        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))), requires_grad=False)

        # ------------------
        #  Train Generators
        # ------------------

        G_AB.train()
        G_BA.train()

        optimizer_G.zero_grad()

        # Identity loss
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)

        loss_identity = (loss_id_A + loss_id_B) / 2

        # GAN loss
        fake_B = G_AB(real_A)
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
        fake_A = G_BA(real_B)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)

        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

        # Cycle loss
        recov_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        recov_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)

        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

        # Total loss
        loss_G = loss_GAN + 10 * loss_cycle + 5 * loss_identity

        loss_G.backward()
        optimizer_G.step()

        # -----------------------
        #  Train Discriminator A
        # -----------------------

        optimizer_D_A.zero_grad()

        # Real loss
        loss_real = criterion_GAN(D_A(real_A), valid)
        # Fake loss (on batch of previously generated samples)
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
        # Total loss
        loss_D_A = (loss_real + loss_fake) / 2

        loss_D_A.backward()
        optimizer_D_A.step()

        # -----------------------
        #  Train Discriminator B
        # -----------------------

        optimizer_D_B.zero_grad()

        # Real loss
        loss_real = criterion_GAN(D_B(real_B), valid)
        # Fake loss (on batch of previously generated samples)
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
        # Total loss
        loss_D_B = (loss_real + loss_fake) / 2

        loss_D_B.backward()
        optimizer_D_B.step()

        loss_D = (loss_D_A + loss_D_B) / 2

        # --------------
        #  Log Progress
        # --------------

        # Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left = total_epoch * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, adv: %f, cycle: %f, identity: %f] ETA: %s"
            % (
                epoch,
                total_epoch,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_GAN.item(),
                loss_cycle.item(),
                loss_identity.item(),
                time_left,
            )
        )

        # If at sample interval save image
        if batches_done % 1000 == 0:
            sample_images(batches_done)

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    torch.save(G_AB.state_dict(), "saved_models/%s/G_AB_%d.pth" % (data_name, epoch))
    torch.save(G_BA.state_dict(), "saved_models/%s/G_BA_%d.pth" % (data_name, epoch))
    torch.save(D_A.state_dict(), "saved_models/%s/D_A_%d.pth" % (data_name, epoch))
    torch.save(D_B.state_dict(), "saved_models/%s/D_B_%d.pth" % (data_name, epoch))


[Epoch 26/50] [Batch 64/3144] [D loss: 0.059493] [G loss: 2.115910, adv: 0.541585, cycle: 0.114435, identity: 0.085995] ETA: 12:10:36.35376073471

KeyboardInterrupt: 

In [13]:
G_AB.load_state_dict(torch.load(glob.glob('./saved_models/monet2photo/G_AB*')[-1]))
G_AB.eval()
my_compose = transforms.Compose(transforms_)

In [14]:
def get_image(image_path) : 
    image = Image.open(image_path)
    image = my_compose(image).reshape(1,3,256,256)
    image = G_AB(torch.FloatTensor(image).cuda())
    image = np.transpose(image[0].cpu().detach().numpy(), (1,2,0))
    image = (image - image.min()) / (image.max() - image.min())
    image = (image * 255).astype(np.uint8)
    return image

In [15]:
image_path = './sample_image/졸업식1.jpg'
image = get_image(image_path)
result = Image.fromarray(image)
result.save('./sample_image/졸업식1_generated.jpg')

In [16]:
image_path = './sample_image/졸업식2.jpg'
image = get_image(image_path)
result = Image.fromarray(image)
result.save('./sample_image/졸업식2_generated.jpg')


In [17]:
image_path = './sample_image/유진이.jpg'
image = get_image(image_path)
result = Image.fromarray(image)
result.save('./sample_image/유진이_generated.jpg')